In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import datetime
import time
import numpy as np
import os
import joblib
import matplotlib.pyplot as plt
from multiprocessing import Pool
from google_drive_downloader import GoogleDriveDownloader as gdd

# Введение

Доброго дня исследователям финансовых рынков!

Предлагаю Вашему вниманию подход к анализу ***индекса S&P 500*** и выработке рекомендательной системы для входа в сделку на инструменте, ассоциированом с этим индексом (допустим, ETF).

Если Вам знакомы термины и инструменты (или даже их практическое применение) как тейкпрофит, стоплосс, волатильность, временное окно, scikit-learn, pandas, tensorflow - смело читайте дальше. Вы обнаружите новые способы анализа финансовых рынков, и, возможно, разовьете идеи дальше.

Весь изложенный в данном материале код и гипотезы, размышления по поводу финансовых рынков - написаны лично. В том числе функции для формирования датасета. Серьезные программисты сочтут код неоптимальным, полагается скидка студенту курсов программирования Python, не имеющему фундаментального образования в сфере ИТ-технологий и без опыта промышленной разработки. Идеи тройного барьерного метода, ситуативного отбора почерпнуты из книг Маркос Лопез де Прадо, методы машинного обучения вдохновлены Ф.Шолле и С.Рашкой. По сути, нас в какой-то мере можно назвать продолжением книг, которые мы читали.

#### Результат, к которому мы стремимся на протяжении всего изложенного материала:
- сформировать ***модель машинного обучения (модель МО)***, которая на основе поминутного анализа каждого компонента S&P 500 будет вырабатывать рекомендации для входа в сделку по S&P 500. Общее рекомендуемое время удержания позиции по сделке - от нескольких часов до нескольких дней, с прибылью по сделке в районе 1 (одного) или нескольких процентов. Напоминаю, что число компонентов индекса - 500.

#### Последовательность шагов, которые будут предприняты для достижения результата

***Часть 1. Загрузка, очистка и первичное форматирование данных.*** Код прост, но данные в разархивированном виде весят более 22 Гб.

>Ввиду объемности входных данных и ресурсоемких вычислений, требовательных к производительности, код до 7 части не рекомендован к запуску на рядовых домашних/офисных компьютерах.

После запуска кода получаем в итоге набор из 500 (или около того) файлов по каждому компоненту индекса S&P 500. Компонент - это отдельный финансовый инструмент, тикер. Каждый файл представляет собой пандас-датафрейм с временным поминутным индексом (период - от 02.01.2008г. до 15.07.2019г.) и двумя столбцами - "High" и "Low". Образец компонента представлен ниже:
![Замещающий текст](pics/2.png)

Как видим, количество строк по одному компоненту - более миллиона. А таких компонентов - 500. Соответственно, идет серьезная нагрузка на вычислительные мощности.

***Часть 2. Словесное-описательное формулирование гипотезы методики сбора датасета.*** В этой части мы пройдем важнейший этап - опишем, на что мы полагаемся, рассчитывая найти с помощью алгоритмов МО закономерности в росте/падении индекса в ближайшие часы. Как же поминутный анализ цены каждой акции поможет нам в этом, и как мы сформируем датасет на основе сочетания гипотезы и данных.

***Часть 3. Функциональная часть кода.*** То есть написание функций, которые путем обращения к скачанным данным сформируют для нас датасет, исходя из логики описательной гипотезы во второй части. Функций немало, и они довольно сложны. Рекомендовано к самостоятельному изучению в случае наличия соответствующей подготовки, и в случае серьезного желания претворения в жизнь изложенного материала. Но пробежаться взглядом и уловить связь между гипотезой и функциями - имеет смысл в любом случае.

Ввиду количества функций и их сложности даны лишь краткие комментарии в начале каждой функции. Подробное описание функций выльется в написание книги, что не является целью данного материала.

***Часть 4. Исполняемая часть кода. Запуск функций для формирования датасета.*** КОД НЕ РЕКОМЕНДОВАН ДЛЯ ЗАПУСКА НА РЯДОВЫХ ДОМАШНИХ КОМПЬЮТЕРАХ. ПЕРВОНАЧАЛЬНЫЕ ДАННЫЕ РАЗМЕРОМ БОЛЕЕ 22 ГБ. ПРОХОДЯТ МНОГОРАЗОВУЮ ИТЕРАТИВНУЮ ОБРАБОТКУ И ПРОЦЕССЫ ВЫЧИСЛЕНИЙ С САМОЙ РАЗНОЙ ЛОГИКОЙ. ПРОЦЕСС ФОРМИРОВАНИЯ ДАТАСЕТА ЧРЕЗВЫЧАЙНО ВЫЧИСЛИТЕЛЬНО ЗАТРАТЕН, К СОЖАЛЕНИЮ НА ДАННЫЙ МОМЕНТ НЕ РАСПАРАЛЛЕЛЕН.

По этой причине образец датасета приложен к данному материалу.
Итак, будет сформирован датасет следующего образца:

In [2]:
dataset = joblib.load('marked_dataset_for_regression_with_high_average_delta.pkl')
dataset

,0_highes3,0_lowes3,0_market_index_rolling,1_highes3,1_lowes3,1_market_index_rolling,2_highes3,2_lowes3,2_market_index_rolling,3_highes3,...,29_highes3,29_lowes3,29_market_index_rolling,30_highes3,30_lowes3,30_market_index_rolling,31_highes3,31_lowes3,31_market_index_rolling,class
2008-01-16 13:00:00,48.0,18.0,-21.177344,20.0,21.0,-27.095469,16.0,55.0,-30.971719,33.0,...,4.0,31.0,-22.636406,25.0,11.0,-20.623750,10.0,27.0,-28.315000,0.330
2008-01-16 14:00:00,9.0,28.0,-23.235156,48.0,18.0,-21.177344,20.0,21.0,-27.095469,16.0,...,57.0,11.0,-11.878281,4.0,31.0,-22.636406,25.0,11.0,-20.623750,14.380
2008-01-16 15:00:00,79.0,19.0,-17.386563,9.0,28.0,-23.235156,48.0,18.0,-21.177344,20.0,...,12.0,41.0,-11.982813,57.0,11.0,-11.878281,4.0,31.0,-22.636406,6.750
2008-01-16 16:00:00,23.0,5.0,-18.121875,79.0,19.0,-17.386563,9.0,28.0,-23.235156,48.0,...,12.0,11.0,-6.479219,12.0,41.0,-11.982813,57.0,11.0,-11.878281,-3.225
2008-01-17 10:00:00,24.0,12.0,-23.029844,23.0,5.0,-18.121875,79.0,19.0,-17.386563,9.0,...,72.0,0.0,4.698281,12.0,11.0,-6.479219,12.0,41.0,-11.982813,0.185
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-06-17 11:00:00,25.0,18.0,7.600625,26.0,30.0,4.577031,44.0,9.0,3.758125,12.0,...,81.0,8.0,52.966094,6.0,14.0,42.940625,19.0,53.0,46.070781,2.980
2019-06-17 12:00:00,23.0,27.0,7.735781,25.0,18.0,7.600625,26.0,30.0,4.577031,44.0,...,16.0,25.0,43.704063,81.0,8.0,52.966094,6.0,14.0,42.940625,-0.860
2019-06-17 13:00:00,14.0,35.0,5.194063,23.0,27.0,7.735781,25.0,18.0,7.600625,26.0,...,34.0,10.0,34.542500,16.0,25.0,43.704063,81.0,8.0,52.966094,3.340
2019-06-17 14:00:00,15.0,19.0,7.247344,14.0,35.0,5.194063,23.0,27.0,7.735781,25.0,...,3.0,54.0,23.994688,34.0,10.0,34.542500,16.0,25.0,43.704063,2.715


***Часть 5. Получение таргет-столбца в виде абсолютного изменения цены***. Будет раскрыт подход к сравнительному анализу цен **`(High+Low)/2`** и **`Close`** для целей машинного обучения, и написан код для получения целевого таргет-столбца. Код запускать не имеет смысла, если не запускался код формирования датасета в предыдущей части.

***Часть 6. Сокращение размерности датасета, и сохранение окончательного датасета для машинного обучения***. Не требует пояснений.

***Часть 7. Задача регрессии. Определение изменения цены HIGH в течении часа***.
Непосредственная работа с образцом датасета. Написанный в этой части код предназначен для многопроцессорной обработки, и обучение нейронной сети на датасете занимает буквально минуты. Код можно запускать на многопроцессорных домашних компьютерах, не опасаясь проблем производительности.

В этой части будет самое интересное - вот такая картинка:
![Замещающий текст](pics/1.png)

Да, это оно. Синим цветом - фактическое движение (точнее, изменение) индекса по часам в тестовом периоде, красным цветом - выдача обученной модели на данных этого периода. То есть тех данных, которые не участвовали в процессе обучения модели. Сложность построения гипотезы и затянутость многофункционального подхода себя оправдывают.

Намеренно избегаю фраз типа **"прогноз" и "модель видит будущую цену"**. В машинном обучении важна точность формулировок для сохранения ясности мышления. Никто ничего не видит, никакого будущего. Обученная модель "находит" закономерности в многомерном пространстве параметров и возможностей на основе точек данных - событий прошлого. Если эти точки (одного класса! например, "цена выросла на 1%") собрались в одной ограниченном подпространстве (естественно, в силу внешних объективных причин), то ничего магического в том, что компьютер нашел это место скопления - нет. И если конфигурация внешних причин не меняется с течением времени - нам повезло, модель работает. Мы можем проверять это регулярно. Если меняется - надо искать дальше, что же конкретно поменялось во внешней среде, и улучшать модель.

Так что, если машинное обучение как бы "видит", какая будет цена через час или два, или день - ничего удивительного в этом нет. Наоборот, отрицая такую возможность, мы отрицаем причинность явлений и логику, и математическое устройство мира. Углубляться не будем, знающие поймут о чем речь.

***Часть 8. Разбор концепции ситуативно-управляемого отбора***. Событийный подход к индексу и к финансовым рынкам вообще. Устранение периодов зашумленности рынков для повышения точности модели.

***Часть 9. Задача классификации. Концепция тройного барьерного метода, с учетом волатильности в момент входа в сделку***. В частях 7 и 8 разбиралась регрессионная задача - какая будет цена в течение ближайшего часа.

В этой же части мы разберем, как эффективно решать насущный вопрос для инвестора - какое действие предпринять в текущий момент? Покупать, продавать, или держать позицию (т.е. ничего не делать)? Типичная задача 3-хклассовой классификации.

Если допустим модель выдает рекомендацию "покупать", то как долго удерживать позицию? В какой момент надо будет "продавать"? Сложная двухступенчатая задача (купить/не купить, если купить то по какой цене продать) решается эффективной методикой **тройного барьерного метода**. В игру вступает волатильность. Подробности в 9-ой части. Код малозатратен, безопасен для запуска.

***Часть 10. Интерпретация полученных результатов и выводы***. Итоговые размышления и анализ результатов.

Итак, поехали.

# Часть 1. Загрузка, очистка и первичное форматирование данных

Скачиваем данные SP&500 в архивированном виде (файл весит более 5 Гб.):

In [3]:
gdd.download_file_from_google_drive(file_id='1DDuIDxeLkRUqR2jkMwgYHl0O_KBgnhPv',
                                    dest_path='./sp500_minutes_2008-2019.zip',
                                    unzip=False)

Если владелец файла удалил его и он недоступен, то доступна следующая ссылка на облаке.мэйл.ру:
https://cloud.mail.ru/public/2A7e/3ynKmoTFV .

Но закачку скорее придется осуществить самостоятельно в графическом интерфейсе браузера, без помощи Python.

После скачивания разархивируем файл в папку  `'./csv_marketdata/'`:

In [4]:
import zipfile
with zipfile.ZipFile('sp500_minutes_2008-2019.zip', 'r') as zip_ref:
    zip_ref.extractall('./csv_marketdata/')

 После разархивирования создаем перечень тикеров-компаний:

In [10]:
path='./csv_marketdata/'
companies = []
for root, dirs, files in os.walk(path):
    for _file in files:
        companies.append(_file)

companies = sorted(companies)[:-1]

In [13]:
len(companies)

500

Пишем функцию для очистки и форматирования данных:

In [14]:
def square(company, path='./csv_marketdata/MarketData/Candles/1Min/'):
    try:
        df = pd.read_csv(path+company, header=None)
        times = [datetime.datetime.strptime(str(df[0][i])[:-2],"%Y%m%d%H%M") for i in range(len(df))]
        df.index=pd.to_datetime(times)
        df.drop(axis=1, columns=[0,1,4,5], inplace=True)
        df.rename(columns={2: 'High', 3: 'Low'}, inplace=True)
        df.index.name='Date'
        df['company']=company[:-4]
        joblib.dump(df, filename='./df_marketdata/'+company[:-4])
    except:
        print(company)
        pass
    return str(company)

 Запускаем мультипроцессорную обработку данных исходя из функции выше.
 Параметры **`agents, chunksize`** настраиваются исходя из характеристик Вашего компьютера:

In [15]:
if __name__ == '__main__':
    agents = 3
    chunksize = 2
    with Pool(processes=agents) as pool:
        result = pool.map(square, companies, chunksize)

#### Отлично, теперь данные готовы для формирования датасета и обучения, и перехода ко второй части.